<a href="https://colab.research.google.com/github/tushant-akar/CS367-Artifical-Intelligence-Lab/blob/main/Marble_Solatire.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import heapq
from typing import List, Tuple
import time

BOARD_SIZE = 7
E, P, X = 0, 1, -1  # Empty, Peg, Invalid

In [2]:
INITIAL_BOARD = [
    [X, X, P, P, P, X, X],
    [X, X, P, P, P, X, X],
    [P, P, P, P, P, P, P],
    [P, P, P, E, P, P, P],
    [P, P, P, P, P, P, P],
    [X, X, P, P, P, X, X],
    [X, X, P, P, P, X, X]
]

MOVES = [(-2, 0), (2, 0), (0, -2), (0, 2)]

In [3]:
class State:
    def __init__(self, board: List[List[int]], pegs: int, moves: List[Tuple[int, int, int, int]]):
        self.board = board
        self.pegs = pegs
        self.moves = moves

    def __lt__(self, other):
        return self.pegs < other.pegs

In [4]:
def is_valid_move(board: List[List[int]], row: int, col: int, move: Tuple[int, int]) -> bool:
    nr, nc = row + move[0], col + move[1]
    mr, mc = row + move[0] // 2, col + move[1] // 2
    return (0 <= nr < BOARD_SIZE and 0 <= nc < BOARD_SIZE and
            board[row][col] == P and board[mr][mc] == P and board[nr][nc] == E)

In [5]:
def make_move(board: List[List[int]], row: int, col: int, move: Tuple[int, int]) -> List[List[int]]:
    new_board = [row[:] for row in board]
    nr, nc = row + move[0], col + move[1]
    mr, mc = row + move[0] // 2, col + move[1] // 2
    new_board[row][col] = new_board[mr][mc] = E
    new_board[nr][nc] = P
    return new_board

In [6]:
def get_successors(state: State) -> List[State]:
    return [State(make_move(state.board, r, c, m), state.pegs - 1, state.moves + [(r, c, r + m[0], c + m[1])])
            for r in range(BOARD_SIZE) for c in range(BOARD_SIZE) for m in MOVES
            if is_valid_move(state.board, r, c, m)]

In [7]:
def is_goal_state(state: State) -> bool:
    return state.pegs == 1

In [8]:
def heuristic1(state: State) -> int:
    return state.pegs - 1

def heuristic2(state: State) -> int:
    return sum(abs(r - 3) + abs(c - 3) for r in range(BOARD_SIZE) for c in range(BOARD_SIZE) if state.board[r][c] == P)

In [9]:
def search(initial_state: State, heuristic, search_type: str) -> Tuple[State, int, int]:
    frontier = [(heuristic(initial_state), 0, initial_state)]
    explored = set()
    nodes_expanded = 0

    while frontier:
        _, g, current_state = heapq.heappop(frontier)

        if is_goal_state(current_state):
            return current_state, nodes_expanded, len(explored)

        board_tuple = tuple(map(tuple, current_state.board))
        if board_tuple in explored:
            continue

        explored.add(board_tuple)
        nodes_expanded += 1

        for successor in get_successors(current_state):
            h = heuristic(successor)
            if search_type == 'a_star':
                f = g + 1 + h
            elif search_type == 'best_first':
                f = h
            else:
                f = successor.pegs
            heapq.heappush(frontier, (f, g + 1, successor))

    return None, nodes_expanded, len(explored)

In [10]:
def print_results(algorithm_name: str, result: Tuple[State, int, int], execution_time: float):
    solution, nodes_expanded, explored_states = result
    if solution:
        print(f"{algorithm_name}:")
        print(f"  Moves: {len(solution.moves)}")
        print(f"  Nodes expanded: {nodes_expanded}")
        print(f"  States explored: {explored_states}")
        print(f"  Execution time: {execution_time:.4f} seconds")
    else:
        print(f"{algorithm_name} did not find a solution.")
    print()

In [11]:
def main():
    initial_state = State(INITIAL_BOARD, sum(row.count(P) for row in INITIAL_BOARD), [])

    algorithms = [
        ("Priority Queue Search", lambda s: search(s, heuristic1, 'priority_queue')),
        ("Best-First Search (h1)", lambda s: search(s, heuristic1, 'best_first')),
        ("Best-First Search (h2)", lambda s: search(s, heuristic2, 'best_first')),
        ("A* Search (h1)", lambda s: search(s, heuristic1, 'a_star')),
        ("A* Search (h2)", lambda s: search(s, heuristic2, 'a_star'))
    ]

    for name, algorithm in algorithms:
        start_time = time.time()
        result = algorithm(initial_state)
        execution_time = time.time() - start_time
        print_results(name, result, execution_time)

In [ ]:
if __name__ == "__main__":
    main()

Priority Queue Search:
  Moves: 31
  Nodes expanded: 11426
  States explored: 11426
  Execution time: 3.4096 seconds

Best-First Search (h1):
  Moves: 31
  Nodes expanded: 11426
  States explored: 11426
  Execution time: 2.2350 seconds

Best-First Search (h2):
  Moves: 31
  Nodes expanded: 139533
  States explored: 139533
  Execution time: 37.2171 seconds

